In [1]:
"""
OBJETIVO : Unir los datos de estaciones, umbrales y precipitaciones.
           Ademas se limpia los datos: Solo C02 y M02, y se verifica que exista las imagenes satelitales
            
"""
Autor='Diego Paredes'

In [2]:
# Leer imagenes satelitales (.nc) 
from netCDF4 import Dataset, num2date

# Graficos y visualizaciones
import cartopy.crs as ccrs
import cartopy.feature as cfw
import matplotlib.pyplot as plt

# Liberias para manejo de datos
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Liberias estandar
import re
import time
import os

In [38]:
from datetime import datetime, timedelta


In [3]:
"""
DEFINIMOS EL PATH DEL PROYECTO 
"""
with open('../../path_base.txt') as f:
    path_base = f.read()
path_base

'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes'

In [4]:
"""
Variables generales
"""
path_imagenes = 'F:/GOES/' 

dsName = 'dsCompletoPP_V2'
listDataset = [f'{path_base}/Archivos/Inicial/{dsName}.csv'] 

#Variables generales
products = ['C07','C08','C13']
times   = ['10','20','30','40','50','00']

In [5]:
"""
Metodos que permiten en un conjunto de estaciones (dataframe), agrega su posicion XO(longitud), XA(latitud)
"""
# Encuentra las longitudes y latitudes
def getMapFile(imagenFile):    
    try:
        ds = Dataset(imagenFile)      
    except:
        print("No se pudo leer los archivos de imagen")
        print(imagenFile)
        return -1,-1

    # obtiene las coordenadas de los pixeles
    lons = ds.variables['longitude'][:].data
    lats = ds.variables['latitude'][:].data            
        
    return lons, lats  

#Busca el valor X en el array, devuelve su posicion
def getPosMap(x,array):    
    pos = -1
    for i in range(len(array)):
        if abs(array[i]-x)<=0.01:
            pos = i
            
    return  pos  

#en el la imagen satelital
def changeOrigenStation(estaciones,imagenFile):
    try:
        station = pd.read_csv(estaciones)   
    except:
        print("No se pudo leer el  archivos de estaciones")
        return False
    
    lo,la = getMapFile(imagenFile)
    
    station['XO'] = station.apply(lambda x: getPosMap(float(x['LON']), lo),axis=1)
    station['XA'] = station.apply(lambda x: getPosMap(float(x['LAT']), la),axis=1)    
    
    
    return station    

In [62]:
#Del dataset guardamos los datos mas importantes en una columna para facilitar su lectura
def obtenerDir(row):
    fecha = row['fecha']
    fecha_dt = datetime.strptime(fecha, "%Y-%m-%d-%H")
    nueva_fecha_dt = fecha_dt + timedelta(hours=5)
    nueva_fecha = nueva_fecha_dt.strftime("%Y-%m-%d-%H")
        
    year, month, day, hour = nueva_fecha.split('-')
    # filename = f'{path_base}comprimido/{year}_{month}_{day}/{hour}/'
    return f"{row['XO']}--{row['XA']}--{nueva_fecha}"

In [50]:
# Devuelve una lista con lo indices que no se encontraron lso archivos y el producto
# Servira para ver si se teinen todas los frames de la fecha
def comprobarFrames(dfOrignial, path_base, products, times, delete=1):    
    #dfOrignial = obtenerDatos(datafile)   
    start_time = time.time()
    
    dfTotal = pd.unique(dfOrignial['fecha'])
    no_fecha = []
    for fecha in dfTotal:
        fecha_dt = datetime.strptime(fecha, "%Y-%m-%d-%H")
        nueva_fecha_dt = fecha_dt + timedelta(hours=5)
        nueva_fecha = nueva_fecha_dt.strftime("%Y-%m-%d-%H")

        year, month, day, hour = fecha.split('-')
        existe = True
        for p in products:
            for t in range(len(times)):             
                filename = f'{path_base}PNG/{nueva_fecha}/{nueva_fecha}_{t}.png'
                try:                    
                    file_size = os.path.getsize(filename)
                    existe = file_size > 4100000
                except: 
                    existe = False
                    break
                
            if not existe:
                break
        if not existe:
            no_fecha.append(fecha)
            
    print(filename)
    if delete:
        antes = len(dfOrignial)
        df2 = dfOrignial[~dfOrignial['fecha'].isin(no_fecha)]
        despues = len(df2)
        print(f'{antes - despues}/{antes} datos eliminados: No se encontraron los archivos de imagenes satelitales')
    else:
        df2 = dfOrignial

    print("Tiempo tomado en verificar datos: %.2fs" % (time.time() - start_time))
    return df2, no_fecha

In [54]:
#Obtiene los datos de precipitacion de un archivo csv, los procesa y los guarda en OTRO archivo CSV
#regresa la cantidad de estaciones sin datos de precipitaciones
def procesarDatos(path_base,p,umbrales=None):  
    start_time = time.time()
    #Obtenemos la informacion de los archivos
    #"Valores"  Contiene los valores de los datos de precipitacion de manera horaria por estacion (codigo)
    #"estaciones" Contiene los daots de cada estacion (cordenadas,codigo,etc)  
    try:        
        dfCompleto = pd.read_csv(p['Fdata'],encoding='latin-1')    
    except:
        print("No se pudo leer el archivos de valores")
        return False
    station = changeOrigenStation(p['Fanalisis'],p['imagenTest'])
    
    values = dfCompleto[dfCompleto['FLAG'].isin(p['siFLAG'])]  
    
    
    #Lista de [nombre,codigo,xo,xa,longitud,latitud,altura,dato,año,mes,dia,hora,flag]
    resultado = []
    
    #Numero filas
    n = len(values.index) 
    print(f'Datos a procesar: {n}')
    
    #Auxiliares
    total = n
    completados = 1 
    
    noStation = {}   
    for i in values.index:
        #Codigo de estacion
        cod = values['CODIGO'][i]        
        st = station[station['Codigo'] == f'X{cod}']
        if st.empty:            
            st = station[station['Nombre'] == values['NOMBRE'][i]]
        if st.empty:
            noStation[cod] = values['NOMBRE'][i]
            
        if not st.empty:       
            flag = values['FLAG'][i]            
        
            #Datos de estacion
            nombre = st['Nombre'].iloc[0]
            xo = st['XO'].iloc[0]
            xa =  st['XA'].iloc[0]
            lat = st['LAT'].iloc[0]
            lon = st['LON'].iloc[0]
            alt = st['ALT'].iloc[0]

            _90 = st['90%'].iloc[0]
            _99 = st['99%'].iloc[0]
            _75 = st['75%'].iloc[0]

            dfUmb = umbrales[umbrales['Codigo']==cod]
            if not dfUmb.empty:                
                umb1 = dfUmb['Umbral1'].iloc[0]
                umb2 = dfUmb['Umbral2'].iloc[0]
            else:
                umb1 = -1
                umb2 = -1

            flagV2 =  values['FLAGV2'][i]
            #Datos de precipitacion
            dato = values['PRECIPITACION'][i]        
            day, month , year = values['FECHA'][i].split('/')   
            hour = values['HORA'][i].split(':')[0]                    

            resultado.append([nombre,cod,xo,xa,lon,lat,alt,dato,_90,_99,_75,umb1,umb2,f'{year}-{month}-{day}-{hour}',flag,flagV2])
            completados = completados + 1
        
        if not completados % 100000:
            progreso = completados/total*100
            print(f'Estaciones con Error: {len(noStation)}')
            f = '{0:.3g}'.format(progreso)
            print(f"Procesando - {f}%")
    
    print(f"Tiempo tomado en procesar {completados}/{n} datos: %.5fs" % (time.time() - start_time))
    start_time = time.time()
    
    #Guardamos en un csv los datos 
    print("Guardando los datos al archivo dataset....")
    df = pd.DataFrame(resultado, columns = ['nombre','codigo','XO','XA','longitud','latitud','altura','dato','90%','99%','75%',
                                            'umb1','umb2','fecha','flag','flagV2'])
    
    df, no_fecha = limpiarDatos(df, p['path_img'], p, read=False)
    
    df.to_csv(f'{path_base}/Archivos/Inicial/{p["dsName"]}.csv', index=False)
    print(f"Tiempo tomado en guardar {completados}datos: %.5fs" % (time.time() - start_time))
    return noStation, df
    

In [55]:
# Filtra por flags, borra datos absurdos (mayor a 401 y menor a 0),
# y verifica que haya imagenes satatelitas para el dato
def limpiarDatos(listNames, path_imagenes, p, read=True):    
    df = []
    start_time = time.time()
    print(f'Se leera los archivos de datasets...')
    if read:
        for name in listNames:
            try:
                df.append(pd.read_csv(name))   
            except:
                print(f'No se pudo leer el archivo {name} de dataset')
                return -1

        if len(df)>1:
            dsCompleto =  pd.concat(df, ignore_index=True) 
        else:
            dsCompleto =  df[0]
    else:
        dsCompleto = listNames
        
        
    print("Tiempo tomado: %.2fs" % (time.time() - start_time))
    print(f'+Cantidad de datos leidos {len(dsCompleto)}')
    
    # Quitamos los NA valores, negativos y mayores a 400
    print(f'\nSe elimnara los valores nulos y dudosos')
    dsCompleto.dropna(subset=['dato'], axis='index', inplace=True)    
    dsCompleto = dsCompleto[dsCompleto['flag']!='ND']
    dsCompleto = dsCompleto[dsCompleto['dato']>=0]
    dsCompleto = dsCompleto[dsCompleto['dato']<401]
    print("Tiempo tomado: %.2fs" % (time.time() - start_time))
    print(f'+Cantidad de datos luego de elimnar nulos {len(dsCompleto)}')
              
    
    # Seleccionamos FLAGS
    if p['siFLAG']:
        dsCompleto = dsCompleto[dsCompleto['flag'].isin(p['siFLAG'])]    
    if p['siFLAGV2']:
        dsCompleto = dsCompleto[dsCompleto['flagV2'].isin(p['siFLAGV2'])]     
        
    # Buscamos imagenes satelitales para lso archivos
    print(f'\nSe buscara las imagenes satelitales para los datos...')
    dfImagenes, no_fecha = comprobarFrames(dsCompleto,path_imagenes, p['products'], p['times'], p['delete'])    
    print("Tiempo tomado: %.2fs" % (time.time() - start_time))        

            
    #Agregamos lso datos de las estaciones al dataset
    print(f'\nSe agregara los datos de las estaciones(cordenadas, umbral)...')
    dfImagenes['imagen'] = dfImagenes.apply(obtenerDir, axis=1)        
    print(f'-------> CANTIDAD FINAL DE DATOS :  {len(dfImagenes)} <--------------')  
    print("Tiempo total: %.2fs" % (time.time() - start_time))
    return shuffle(dfImagenes), no_fecha

In [53]:
#df = pd.read_csv('C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/FinalTesis/Tesis2-DiegoParedes/Archivos/Dataset/Clasificacion/ClaseV4_DUD_umb1_CompletoDS.csv')
#len(df)

In [63]:
%%time
params = {
    # Archivos
    'Festaciones': f'{path_base}/Archivos/Inicial/estaciones.csv',
    'Fdata'      : f'{path_base}/Archivos/Dataset/dsFLAGV2.csv',
    'imagenTest' : 'D:/GOES/C08/2020/01/G16_C08_Cyl_20200101-0110.nc',
    'Fanalisis'  : f'{path_base}/Archivos/Reportes/Datos/analisis_Total-F2.csv',
    
    # Flags 
    'siFLAG'     : ['C0000002', 'M0000002'],
    'siFLAGV2'   : [],
    'dsName'     : 'GTIME_Paper',
    
    # Imagenes Satelitels
    'path_img'   :  path_imagenes, 
    'products'   :  products,
    'times'      :  times,
    'delete'     :  True
}

"""
FLAGV2 
"""
dfUmbrales = pd.read_csv(f'{path_base}/Archivos/Reportes/Datos/analisis_umbrales.csv')
noStationFV2, datosFLAGV2 = procesarDatos(path_base,params,umbrales=dfUmbrales)

#datosFLAGV2 = pd.read_csv(f'{path_base}/Archivos/Inicial/{params["dsName"]}.csv')
#datosFLAGV2.head(4)

Datos a procesar: 66911
Tiempo tomado en procesar 45491/66911 datos: 42.73729s
Guardando los datos al archivo dataset....
Se leera los archivos de datasets...
Tiempo tomado: 0.00s
+Cantidad de datos leidos 45490

Se elimnara los valores nulos y dudosos
Tiempo tomado: 0.02s
+Cantidad de datos luego de elimnar nulos 45262

Se buscara las imagenes satelitales para los datos...
F:/GOES/PNG/2020-12-24-07/2020-12-24-07_5.png
2417/45262 datos eliminados: No se encontraron los archivos de imagenes satelitales
Tiempo tomado en verificar datos: 18.43s
Tiempo tomado: 18.46s

Se agregara los datos de las estaciones(cordenadas, umbral)...


C:\Users\Shounen\AppData\Local\Temp\ipykernel_30408\4073903732.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfImagenes['imagen'] = dfImagenes.apply(obtenerDir, axis=1)


-------> CANTIDAD FINAL DE DATOS :  42845 <--------------
Tiempo total: 19.94s
Tiempo tomado en guardar 45491datos: 20.48005s
CPU times: total: 49.1 s
Wall time: 1min 3s


In [65]:
datosFLAGV2.to_csv(f'{path_base}/Archivos/Dataset/GTMTime_Base.csv')

In [56]:
df = pd.read_csv(f'{path_base}/Archivos/Dataset/dsFLAGV2.csv')
df = df[(df['PRECIPITACION']>=0) & (df['PRECIPITACION']<401)]


In [31]:
df['fecha_full'] = df['FECHA'].astype(str) +' ' +  df['HORA'].astype(str)
df['fecha_full'] = pd.to_datetime(df['fecha_full'], format='%d/%m/%Y %H:%M:%S')

In [32]:
df['fecha'] = df['fecha_full'].dt.strftime('%Y-%m-%d-%H')

In [47]:
dfLimpio, no_fecha = comprobarFrames(df, path_imagenes, products, times, delete=1)

F:/GOES/PNG/2020-09-12-20/2020-09-12-20_5.png
507613/3016245 datos eliminados: No se encontraron los archivos de imagenes satelitales
Tiempo tomado en verificar datos: 14.61s


In [49]:
dfLimpio.to_csv(f'{path_base}/Archivos/Inicial/GTMTime_Base.csv')